In [25]:
import pandas as pd
from pandas import DataFrame
import numpy as np

### 处理丢失数据
- 有两种丢失数据：
    - None
    - np.nan(NaN)

- 两种丢失数据的区别

In [26]:
type(None)

NoneType

In [27]:
type(np.nan)

float

- 为什么在数据分析中需要用到的是浮点类型的空而不是对象类型？
    - 数据分析中会常常使用某些形式的运算来处理原始数据，如果原数数据中的空值为NAN的形式，则不会干扰或者中断运算。
    - NAN可以参与运算的
    - None是不可以参与运算

In [28]:
np.nan + 1

nan

In [29]:
# None + 1

- 在pandas中如果遇到了None形式的空值则pandas会将其强转成NAN的形式。

In [30]:
df = DataFrame(data=np.random.randint(0,100,size=(7,5)))
df.iloc[2,3] = None
df.iloc[4,2] = np.nan
df.iloc[5,4] = None
df

,0,1,2,3,4
0,75,98,21.0,89.0,0.0
1,80,94,69.0,3.0,65.0
2,0,92,35.0,NaN,98.0
3,92,74,31.0,89.0,70.0
4,71,39,NaN,62.0,27.0
5,15,91,15.0,98.0,NaN
6,51,33,36.0,14.0,58.0


### pandas处理空值操作
- isnull
- notnull
- any
- all
- dropna
- fillna

- 方式1：对空值进行过滤（删除空所在的行数据）
    - 技术：isnull，notnull，any，all

In [31]:
df.isnull()

,0,1,2,3,4
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,True,False
3,False,False,False,False,False
4,False,False,True,False,False
5,False,False,False,False,True
6,False,False,False,False,False


In [32]:
df.notnull().all(axis=1)

0     True
1     True
2    False
3     True
4    False
5    False
6     True
dtype: bool

In [33]:
# 哪些行中有空值
# any(axis=1)检测哪些行中存有空值
df.isnull().any(axis=1) # any会作用isnull返回结果的每一行
# true对应的行就是存有缺失数据的行

0    False
1    False
2     True
3    False
4     True
5     True
6    False
dtype: bool

In [34]:
# 获取空对应的行数据
df.loc[df.isnull().any(axis=1)]
# 获取空对应行数据的行索引
indexs = df.loc[df.isnull().any(axis=1)].index
indexs

Int64Index([2, 4, 5], dtype='int64')

In [35]:
df.drop(labels=indexs,axis=0)

,0,1,2,3,4
0,75,98,21.0,89.0,0.0
1,80,94,69.0,3.0,65.0
3,92,74,31.0,89.0,70.0
6,51,33,36.0,14.0,58.0


- 方式2：
    - dropna：可以直接将缺失的行或者列进行删除

In [36]:
df.dropna(axis=0)

,0,1,2,3,4
0,75,98,21.0,89.0,0.0
1,80,94,69.0,3.0,65.0
3,92,74,31.0,89.0,70.0
6,51,33,36.0,14.0,58.0


- 对缺失值进行覆盖
    - fillna

In [37]:
df.fillna(value=999) # 使用指定值将源数据中所有的空值进行填充

,0,1,2,3,4
0,75,98,21.0,89.0,0.0
1,80,94,69.0,3.0,65.0
2,0,92,35.0,999.0,98.0
3,92,74,31.0,89.0,70.0
4,71,39,999.0,62.0,27.0
5,15,91,15.0,98.0,999.0
6,51,33,36.0,14.0,58.0


In [ ]:
# 使用空的近邻值进行填充
# method=ffill向前填充，bfill向后填充
df.fillna(axis=0, method="bfill")

,0,1,2,3,4
0,75,98,21.0,89.0,0.0
1,80,94,69.0,3.0,65.0
2,0,92,35.0,89.0,98.0
3,92,74,31.0,89.0,70.0
4,71,39,15.0,62.0,27.0
5,15,91,15.0,98.0,58.0
6,51,33,36.0,14.0,58.0


- 什么时候用dropna什么时候用fillna
    - 尽量使用dropna，如果删除成本比较高，则使用fillna

- 使用空值对应列的均值进行空值填充

In [40]:
for col in df.columns:
    # 检测哪些列中存有空值
    if df[col].isnull().sum > 0: # 说明df[col]中存有空值
        mean_value = df[col].mean()
        df[col] = df[col].fillna(value=mean_value)

TypeError: '>' not supported between instances of 'method' and 'int'

### 面试题
- 数据说明： 
    - 数据是1个冷库的温度数据，1-7对应7个温度采集设备，1分钟采集一次。

- 数据处理目标：
    - 用1-4对应的4个必须设备，通过建立冷库的温度场关系模型，预估出5-7对应的数据。
    - 最后每个冷库中仅需放置4个设备，取代放置7个设备。
    - f(1-4) --> y(5-7)

- 数据处理过程：
    - 1、原始数据中有丢帧现象，需要做预处理；
    - 2、matplotlib 绘图；
    - 3、建立逻辑回归模型。

- 无标准答案，按个人理解操作即可，请把自己的操作过程以文字形式简单描述一下，谢谢配合。

- 测试数据为testData.xlsx


In [47]:
data = pd.read_excel("../data/testData.xlsx").drop(labels=['none','none1'],axis=1)
data

/usr/local/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,time,1,2,3,4,5,6,7
0,2019-01-27 17:00:00,-24.8,-18.2,-20.8,-18.8,NaN,NaN,NaN
1,2019-01-27 17:01:00,-23.5,-18.8,-20.5,-19.8,-15.2,-14.5,-16.0
2,2019-01-27 17:02:00,-23.2,-19.2,NaN,NaN,-13.0,NaN,-14.0
3,2019-01-27 17:03:00,-22.8,-19.2,-20.0,-20.5,NaN,-12.2,-9.8
4,2019-01-27 17:04:00,-23.2,-18.5,-20.0,-18.8,-10.2,-10.8,-8.8
...,...,...,...,...,...,...,...,...
1055,2019-01-28 10:35:00,-26.2,-27.2,-28.8,-27.5,-2.0,NaN,-5.0
1056,2019-01-28 10:36:00,-26.8,-27.5,-29.0,-27.8,-2.2,NaN,-5.0
1057,2019-01-28 10:37:00,-27.2,-27.8,-29.0,-28.0,-2.2,NaN,-5.0
1058,2019-01-28 10:38:00,-27.5,-27.0,-29.0,-28.0,-3.5,-3.2,-5.8


In [48]:
data.shape

(1060, 8)

In [49]:
# 删除空对应的行数据
data.dropna(axis=0).shape

(927, 8)

In [60]:
# 填充
data = data.fillna(method='ffill',axis=0).fillna(method='bfill',axis=0)

In [61]:
data.isnull().any(axis=1)

0       False
1       False
2       False
3       False
4       False
        ...  
1055    False
1056    False
1057    False
1058    False
1059    False
Length: 1060, dtype: bool

### 处理重复数据

In [75]:
df = DataFrame(data=np.random.randint(0,100,size=(8,6)))
df.iloc[1] = [1,1,1,1,1,1]
df.iloc[3] = [1,1,1,1,1,1]
df.iloc[5] = [1,1,1,1,1,1]
df

,0,1,2,3,4,5
0,94,40,52,92,98,48
1,1,1,1,1,1,1
2,43,17,84,5,5,71
3,1,1,1,1,1,1
4,38,75,8,3,18,38
5,1,1,1,1,1,1
6,0,62,46,58,94,72
7,70,66,50,22,77,98


In [77]:
# 检测哪些行存有重复的数据
df.duplicated(keep="first")

0    False
1    False
2    False
3     True
4    False
5     True
6    False
7    False
dtype: bool

In [54]:
df.loc[~df.duplicated(keep="first")]

,0,1,2,3,4,5
0,41,31,8,75,42,16
1,1,1,1,1,1,1
2,97,33,7,83,67,90
4,86,59,68,75,71,32
6,39,7,22,42,88,92
7,61,57,53,17,16,82


In [ ]:
# 一步到位删除
df.drop_duplicates(keep="first")

,0,1,2,3,4,5
0,41,31,8,75,42,16
1,1,1,1,1,1,1
2,97,33,7,83,67,90
4,86,59,68,75,71,32
6,39,7,22,42,88,92
7,61,57,53,17,16,82


### 处理异常数据
- 自定义一个1000行3列（A，B，C）取值范围为0-1的数据源，然后将C列中的值大于其两倍标准差的异常值进行清洗

In [69]:
df = DataFrame(data=np.random.random(size=(1000,3)),columns=['A','B','C'])
df.shape

(1000, 3)

In [70]:
# 制定判定异常值的条件
twice_std = df["C"].std() * 2
twice_std

0.5758080007105679

In [71]:
df.loc[~(df["C"] > twice_std)]

,A,B,C
0,0.433369,0.740087,0.332691
3,0.081504,0.073904,0.519129
4,0.971252,0.515482,0.440797
6,0.756666,0.736717,0.337477
8,0.498912,0.818301,0.522315
...,...,...,...
993,0.282138,0.715351,0.565837
994,0.823675,0.260397,0.534165
996,0.476512,0.807161,0.572066
997,0.299960,0.241353,0.318832
